- 策略更新比率（ratio）：$r(\theta)=\frac{\pi_\theta(a|s)}{\pi_{\theta_\text{old}}(a|s)}$
- Advantage（优势函数）本身不直接参与梯度计算
- PPO 的 clip 操作，会导致这条数据没有梯度，这条训练数据就起不到贡献了
    - $r(\theta)$ 不在区间 $[1-\epsilon,1+\epsilon]$ 时；

### TRL ppotrainer

- https://github.com/huggingface/trl/blob/main/trl/trainer/ppo_trainer.py
- make experiences
    - forward
        - queries: `[4, 56]`
        - reponses: `[4, 53]`（$\pi_{\theta_{old}}$）
        - logprobs: `[4, 53]` （$\pi_{\theta_{old}}$）
        - ref_logprobs: `[4, 53]`（$\pi_{ref}$）
        - values: `[4, 53]`
        - scores: `[4]` (last token's, the whole query + reponse)
    - 计算 rewards (token 级别)
        - $r_t = r_{T} - \beta (\log\pi_\theta-\log\pi_{ref})$
            - 内循环；
            - KL 项是 k1 近似；
    - 计算 advantage & return
        - GAE：
            - $\delta_t=r_t+\gamma V(s_{t+1})-V(s_t)$
            - $A_t=\sum_{k=0}^T(\gamma\lambda)^k\delta_{t+k}$
        - return: advantage + value
- ppo update ($\pi_\theta$)